In [1]:
from ortools.init.python import init

In [ ]:
import requests
import os

API_KEY = os.getenv("GPLACES_API_KEY")

PLACES_SEARCH_URL = "https://places.googleapis.com/v1/places:searchText"

def get_places_lat_lng(query: str, api_key: str = API_KEY, limit: int = 10):
    """
    Get a list of places (name + lat,lng + Google Maps link) from Places API v1 using a query.
    
    Args:
        query (str): Search query like "temples in Bangalore".
        api_key (str): Google Cloud API Key.
        limit (int): Number of places to return (default 10).
    
    Returns:
        list of dict with name, lat, lng, location_link, and place_id
    """
    url = PLACES_SEARCH_URL
    
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.location,places.id"
    }
    
    body = {
        "textQuery": query,
        "pageSize": limit
    }
    
    resp = requests.post(url, headers=headers, json=body)
    resp.raise_for_status()
    
    data = resp.json()
    results = []
    
    for place in data.get("places", []):
        results.append({
            "name": place["displayName"]["text"],
            "address": place.get("formattedAddress"),
            "lat": place["location"]["latitude"],
            "lng": place["location"]["longitude"],
            "place_id": place["id"],
            "location_link": f"https://www.google.com/maps/search/?api=1&query={place['location']['latitude']},{place['location']['longitude']}"
        })
    
    return results

ROUTES_ENDPOINT = "https://routes.googleapis.com/distanceMatrix/v2:computeRouteMatrix"

def build_distance_matrix(places: list, api_key: str = API_KEY, travel_mode: str = "DRIVE"):
    """
    Build a distance/time matrix between all given places using Google Routes API v2.
    
    Args:
        places (list): List of dicts with at least {"lat", "lng"} (from get_places_lat_lng).
        api_key (str): Google Cloud API Key.
        travel_mode (str): "DRIVE", "WALK", "BICYCLE", "TRANSIT" (default DRIVE).
    
    Returns:
        dict with distance_matrix[origin_index][destination_index] = {distance_meters, duration_seconds}
    """
    
    url = ROUTES_ENDPOINT
    
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "originIndex,destinationIndex,duration,distanceMeters"
    }
    
    origins = [{"waypoint": {"location": {"latLng": {"latitude": p["lat"], "longitude": p["lng"]}}}} for p in places]
    destinations = [{"waypoint": {"location": {"latLng": {"latitude": p["lat"], "longitude": p["lng"]}}}} for p in places]
    
    body = {
        "origins": origins,
        "destinations": destinations
    }
    # print(json.dumps(body))
    
    resp = requests.post(url, headers=headers,  json=body)
    resp.raise_for_status()
    
    data = resp.json()
    print(data)
    
    # Initialize matrix
    n = len(places)
    matrix = [[None] * n for _ in range(n)]
    
    for row in data:
        o = row["originIndex"]
        d = row["destinationIndex"]

        # duration may come as "123s" (string)
        duration_str = row.get("duration", "0s")
        duration_seconds = int(duration_str.replace("s", "")) if isinstance(duration_str, str) else 0

        matrix[o][d] = {
            "distance_meters": row.get("distanceMeters", 0),
            "duration_seconds": duration_seconds
        }
    
    return matrix




In [3]:
places = get_places_lat_lng("tourist attractions near goa")

In [4]:
places

[{'name': 'Fort Aguada',
  'address': 'Fort Aguada Rd, Aguada Fort Area, Candolim, Goa 403515, India',
  'lat': 15.492251900000001,
  'lng': 73.77374619999999,
  'place_id': 'ChIJa72MxnXBvzsRHHtpszB2g6M',
  'location_link': 'https://www.google.com/maps/search/?api=1&query=15.492251900000001,73.77374619999999'},
 {'name': 'Dudhsagar Falls',
  'address': 'Sonauli, Goa 403410, India',
  'lat': 15.314437499999997,
  'lng': 74.3143073,
  'place_id': 'ChIJQ4srFBimvzsRJQI7KOdIlP0',
  'location_link': 'https://www.google.com/maps/search/?api=1&query=15.314437499999997,74.3143073'},
 {'name': 'Harvalem Waterfalls',
  'address': 'Rudreshwar Colony, Kudne, Goa 403505, India',
  'lat': 15.5507728,
  'lng': 74.0264703,
  'place_id': 'ChIJVUbAVEy9vzsRS6uKBv4zNmI',
  'location_link': 'https://www.google.com/maps/search/?api=1&query=15.5507728,74.0264703'},
 {'name': 'Velsao Beach',
  'address': '9V3M+QG9, Consua, Velsao, Goa 403712, India',
  'lat': 15.3544232,
  'lng': 73.88386369999999,
  'place_id

In [5]:

try:
    matrix = build_distance_matrix(places)
    matrix
except Exception as e:
    print(e)

[{'originIndex': 4, 'destinationIndex': 4, 'duration': '0s'}, {'originIndex': 9, 'destinationIndex': 9, 'duration': '0s'}, {'originIndex': 0, 'destinationIndex': 0, 'duration': '0s'}, {'originIndex': 3, 'destinationIndex': 3, 'duration': '0s'}, {'originIndex': 9, 'destinationIndex': 8, 'distanceMeters': 8087, 'duration': '1606s'}, {'originIndex': 7, 'destinationIndex': 7, 'duration': '0s'}, {'originIndex': 8, 'destinationIndex': 8, 'duration': '0s'}, {'originIndex': 1, 'destinationIndex': 1, 'duration': '0s'}, {'originIndex': 7, 'destinationIndex': 0, 'distanceMeters': 3508, 'duration': '594s'}, {'originIndex': 3, 'destinationIndex': 5, 'distanceMeters': 8044, 'duration': '880s'}, {'originIndex': 8, 'destinationIndex': 9, 'distanceMeters': 8152, 'duration': '1593s'}, {'originIndex': 6, 'destinationIndex': 9, 'distanceMeters': 19467, 'duration': '1876s'}, {'originIndex': 2, 'destinationIndex': 2, 'duration': '0s'}, {'originIndex': 5, 'destinationIndex': 3, 'distanceMeters': 8055, 'durat

In [6]:
matrix

[[{'distance_meters': 0, 'duration_seconds': 0},
  {'distance_meters': 84619, 'duration_seconds': 8413},
  {'distance_meters': 42576, 'duration_seconds': 4718},
  {'distance_meters': 37796, 'duration_seconds': 3908},
  {'distance_meters': 17084, 'duration_seconds': 2815},
  {'distance_meters': 32815, 'duration_seconds': 3219},
  {'distance_meters': 99164, 'duration_seconds': 8883},
  {'distance_meters': 3508, 'duration_seconds': 625},
  {'distance_meters': 82354, 'duration_seconds': 8139},
  {'distance_meters': 82584, 'duration_seconds': 7453}],
 [{'distance_meters': 86531, 'duration_seconds': 8492},
  {'distance_meters': 0, 'duration_seconds': 0},
  {'distance_meters': 56199, 'duration_seconds': 5730},
  {'distance_meters': 67767, 'duration_seconds': 6625},
  {'distance_meters': 89978, 'duration_seconds': 9081},
  {'distance_meters': 61310, 'duration_seconds': 6102},
  {'distance_meters': 68389, 'duration_seconds': 7051},
  {'distance_meters': 84180, 'duration_seconds': 8238},
  {'dis

In [11]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def solve_tsp_with_fixed_start_end(distance_matrix, start_index, end_index=None):
    """
    Solve TSP with OR-Tools given a distance matrix.
    Fix start, optionally fix end.

    Args:
        distance_matrix (list[list[dict]]): NxN matrix where matrix[i][j]["duration_seconds"] or ["distance_meters"].
        start_index (int): index of start node (e.g., airport).
        end_index (int or None): index of end node. If None, same as start.

    Returns:
        (route, route_distance): route is a list of indices.
    """

    size = len(distance_matrix)

    # Convert to plain int cost matrix
    cost_matrix = [
        [distance_matrix[i][j]["duration_seconds"] if distance_matrix[i][j] else 999999
         for j in range(size)]
        for i in range(size)
    ]

    # --- FIX: use starts/ends as lists ---
    if end_index is None:
        end_index = start_index
    manager = pywrapcp.RoutingIndexManager(size, 1, [start_index], [end_index])

    routing = pywrapcp.RoutingModel(manager)

    # Callback to return travel time
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return cost_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(5)

    # Solve
    solution = routing.SolveWithParameters(search_parameters)
    if not solution:
        return None, None

    # Extract route
    route = []
    index = routing.Start(0)
    route_distance = 0

    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(node)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)

    route.append(manager.IndexToNode(index))  # add end

    return route, route_distance


In [12]:
route, total_time = solve_tsp_with_fixed_start_end(matrix, 0, 1)

print("Optimal route:")
for idx in route:
    print(f"- {places[idx]['name']} ({places[idx]['location_link']})")

print(f"Total travel time: {total_time/3600:.2f} hours")

Optimal route:
- Fort Aguada (https://www.google.com/maps/search/?api=1&query=15.492251900000001,73.77374619999999)
- Sinquerim Fort (https://www.google.com/maps/search/?api=1&query=15.4984466,73.7663559)
- Chapora Fort (https://www.google.com/maps/search/?api=1&query=15.6046375,73.7369631)
- Harvalem Waterfalls (https://www.google.com/maps/search/?api=1&query=15.5507728,74.0264703)
- Kesarval Spring Verna Waterfall (https://www.google.com/maps/search/?api=1&query=15.382311900000001,73.92880099999999)
- Velsao Beach (https://www.google.com/maps/search/?api=1&query=15.3544232,73.88386369999999)
- Butterfly Beach Goa (https://www.google.com/maps/search/?api=1&query=15.019580099999999,74.001638)
- Palolem Beach, GOA (https://www.google.com/maps/search/?api=1&query=15.0083261,74.0251467)
- Bamanbudo Waterfall (https://www.google.com/maps/search/?api=1&query=15.060051500000002,74.1595025)
- Dudhsagar Falls (https://www.google.com/maps/search/?api=1&query=15.314437499999997,74.3143073)
Total